In [2]:
import numpy as np
import pickle
import math
import matplotlib.pyplot as plt
import pandas as pd
import time
import pprint
import seaborn as sns
import CMR_IA as cmr
sns.set_context('paper')
# pd.set_option('display.max_columns', None)

In [3]:
# association recognition - cued recall
with open("../Data/simuS2_design.pkl", 'rb') as inp:
    df_study = pickle.load(inp)
    df_test = pickle.load(inp)
df_study = df_study.loc[df_study.session < 100]
df_test = df_test.loc[df_test.session < 100]

In [4]:
df_study

,study_itemno1,study_itemno2,study_item1,study_item2,session,pair_idx
0,1622,811,WORLD,LENS,0,8
1,897,346,MINK,COLLEGE,0,19
2,569,828,FIREMAN,LIP,0,0
3,1077,674,PIT,GYM,0,21
4,1422,1103,SWITCH,POOL,0,13
...,...,...,...,...,...,...
14395,829,269,LIPSTICK,CEMETERY,99,16
14396,8,2,ADULT,ACID,99,1
14397,408,182,CRICKET,BRUSH,99,18
14398,288,946,CHEERLEADER,NET,99,13


In [5]:
df_test

,test_itemno1,test_itemno2,test_item1,test_item2,correct_ans,pair_idx,type,test,session
0,98,142,BATH,BLUEPRINT,0,-1,extra,1,0
1,1309,1083,SINK,PLANT,0,-1,extra,1,0
2,1058,-1,PICK,ZOO,0,-1,extra,1,0
3,754,-1,JELLO,ZOO,1,24,Same Item,1,0
4,1601,1409,WHALE,SUPERMARKET,1,6,Intact Pair,1,0
...,...,...,...,...,...,...,...,...,...
11995,1002,-1,PAIL,ZOO,1,22,Same Item,2,99
11996,984,-1,ORGAN,ZOO,1,9,Different Item,2,99
11997,1344,272,SOLDIER,CEREAL,1,7,Intact Pair,2,99
11998,430,161,CUSTOMER,BOY,1,24,Item-Pair,2,99


### Run CMR-IA

In [6]:
s_mat = np.load('../Data/wordpools/ltp_FR_similarity_matrix.npy')

In [7]:
params = cmr.make_default_params()
params.update(
    beta_enc = 0.5,
    beta_cue = 0.5,
    beta_rec = 0,
    beta_rec_new = 0.1, # beta for retrieved item
    beta_rec_post = 0.1,
    gamma_fc = 0.5,
    gamma_cf = 0.5,
    s_fc = 0.1,
    s_cf = 0.1,
    c_thresh = 0.3,
    c_thresh_ass = 0.70,
    a = 2800,
    b = 2.14,
    m = 0,
    n = 1,
    kappa = 0.02,
    lamb = 0,
    eta = 0.01,
    phi_s = 1,
    phi_d = 0.6,
    nitems_in_accumulator = 96,
    d_ass = 1,
    learn_while_retrieving = True,
    omega = 2,
    alpha = 1
)
params

{'beta_enc': 0.5,
 'beta_rec': 0,
 'beta_cue': 0.5,
 'beta_rec_new': 0.1,
 'beta_rec_post': 0.1,
 'beta_distract': None,
 'phi_s': 1,
 'phi_d': 0.6,
 's_cf': 0.1,
 's_fc': 0.1,
 'kappa': 0.02,
 'eta': 0.01,
 'omega': 2,
 'alpha': 1,
 'c_thresh': 0.3,
 'c_thresh_ass': 0.7,
 'd_ass': 1,
 'lamb': 0,
 'rec_time_limit': 60000.0,
 'dt': 10,
 'nitems_in_accumulator': 96,
 'max_recalls': 50,
 'learn_while_retrieving': True,
 'a': 2800,
 'b': 2.14,
 'm': 0,
 'n': 1,
 'c1': 0,
 'No_recall': None,
 'var_enc': 1,
 'bad_enc_ratio': 1,
 'gamma_fc': 0.5,
 'gamma_cf': 0.5}

In [8]:
df_simu, f_in_acc, f_in_dif = cmr.run_success_multi_sess(params,df_study, df_test, s_mat, mode="Recog-Recog")
df_simu

CMR2 Time: 7.3387298583984375


,session,test_itemno1,test_itemno2,s_resp,s_rt,csim
0,0,98,142,0.0,1666.318726,0.057475
1,0,1309,1083,0.0,1713.367188,0.070487
2,0,1058,-1,0.0,1745.471558,0.079161
3,0,754,-1,0.0,1960.408569,0.133427
4,0,1601,1409,0.0,1442.324707,0.609983
...,...,...,...,...,...,...
11995,99,1002,-1,1.0,1417.171631,0.618204
11996,99,984,-1,1.0,1319.988525,0.651400
11997,99,1344,272,1.0,1018.223083,0.772692
11998,99,430,161,0.0,2022.117676,0.147909


In [9]:
df_simu['test'] = df_test['test']
df_simu = df_simu.merge(df_test,on=['session','test','test_itemno1','test_itemno2'])
df_simu['correct'] = df_simu.s_resp == df_simu.correct_ans
df_simu

,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,pair_idx,type,correct
0,0,98,142,0.0,1666.318726,0.057475,1,BATH,BLUEPRINT,0,-1,extra,True
1,0,1309,1083,0.0,1713.367188,0.070487,1,SINK,PLANT,0,-1,extra,True
2,0,1058,-1,0.0,1745.471558,0.079161,1,PICK,ZOO,0,-1,extra,True
3,0,754,-1,0.0,1960.408569,0.133427,1,JELLO,ZOO,1,24,Same Item,False
4,0,1601,1409,0.0,1442.324707,0.609983,1,WHALE,SUPERMARKET,1,6,Intact Pair,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,99,1002,-1,1.0,1417.171631,0.618204,2,PAIL,ZOO,1,22,Same Item,True
11996,99,984,-1,1.0,1319.988525,0.651400,2,ORGAN,ZOO,1,9,Different Item,True
11997,99,1344,272,1.0,1018.223083,0.772692,2,SOLDIER,CEREAL,1,7,Intact Pair,True
11998,99,430,161,0.0,2022.117676,0.147909,2,CUSTOMER,BOY,1,24,Item-Pair,False


In [10]:
df_target = df_simu.query("type != 'extra'")
df_target['studied'] = df_target.apply(lambda x: True if x['pair_idx'] < 25 else False, axis = 1)
df_target

C:\Users\beige\AppData\Local\Temp\ipykernel_20344\1487575911.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_target['studied'] = df_target.apply(lambda x: True if x['pair_idx'] < 25 else False, axis = 1)


,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,pair_idx,type,correct,studied
3,0,754,-1,0.0,1960.408569,0.133427,1,JELLO,ZOO,1,24,Same Item,False,True
4,0,1601,1409,0.0,1442.324707,0.609983,1,WHALE,SUPERMARKET,1,6,Intact Pair,False,True
7,0,916,720,0.0,2213.085449,0.190079,1,MOP,HOSE,0,25,Pair-Item,True,False
11,0,346,-1,0.0,2638.558838,0.272249,1,COLLEGE,ZOO,1,19,Different Item,False,True
14,0,1027,773,0.0,1520.254028,0.585393,1,PATROL,KEY,1,4,Intact Pair,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11994,99,209,488,0.0,1291.768677,0.661499,2,CAB,DORM,1,5,Item-Pair,False,True
11995,99,1002,-1,1.0,1417.171631,0.618204,2,PAIL,ZOO,1,22,Same Item,True,True
11996,99,984,-1,1.0,1319.988525,0.651400,2,ORGAN,ZOO,1,9,Different Item,True,True
11997,99,1344,272,1.0,1018.223083,0.772692,2,SOLDIER,CEREAL,1,7,Intact Pair,True,True


In [12]:
df_target.groupby(["type","studied"]).s_resp.mean()

type            studied
Different Item  False      0.000000
                True       0.932673
Intact Pair     False      0.000000
                True       0.239754
Item-Pair       False      0.000000
                True       0.579921
Pair-Item       False      0.000000
                True       0.579430
Same Item       False      0.000000
                True       0.912525
Name: s_resp, dtype: float64

In [14]:
def get_pair(df_tmp):
    df_tmp_pair = pd.pivot_table(df_tmp,index=["pair_idx","type"],columns="test",values="correct")
    df_tmp_pair.columns = ["test1","test2"]
    df_tmp_pair.reset_index(inplace=True)
    return df_tmp_pair

df_simu_p = df_target.query("pair_idx >= 0")
df_pair = df_simu_p.groupby("session").apply(get_pair).reset_index()
df_pair

,session,level_1,pair_idx,type,test1,test2
0,0,0,0,Item-Pair,1.0,0.0
1,0,1,1,Pair-Item,0.0,1.0
2,0,2,2,Same Item,1.0,0.0
3,0,3,3,Pair-Item,0.0,1.0
4,0,4,4,Intact Pair,0.0,0.0
...,...,...,...,...,...,...
2995,99,25,25,Item-Pair,1.0,1.0
2996,99,26,26,Pair-Item,1.0,1.0
2997,99,27,27,Pair-Item,1.0,1.0
2998,99,28,28,Pair-Item,1.0,1.0


In [22]:
types = np.unique(df_pair.type)
for type in types:
    print(type)
    
    df_tmp = df_pair.loc[df_pair.type == type]
    df_tab = pd.crosstab(index=df_tmp.test2,columns=df_tmp.test1,normalize=False)
    print(df_tab)

    def Yule_Q(A, B, C, D):
        return (A * D - B * C) / (A * D + B * C)
    
    print("Q: ", Yule_Q(df_tab[1][1]+0.5,df_tab[0][1]+0.5,df_tab[1][0]+0.5,df_tab[0][0]+0.5))

Different Item
test1  0.0  1.0
test2          
0.0     14   26
1.0     14  546
Q:  0.9075043630017452
Intact Pair
test1  0.0  1.0
test2          
0.0    163   25
1.0     20   92
Q:  0.9331799443965104
Item-Pair
test1  0.0  1.0
test2          
0.0     48  533
1.0      7  312
Q:  0.5822784810126582
Pair-Item
test1  0.0  1.0
test2          
0.0     39    5
1.0    330  226
Q:  0.6622694969575921
Same Item
test1  0.0  1.0
test2          
0.0     26   13
1.0     23  538
Q:  0.9565038560411311
